# Метод опорных векторов

1. Построить модель опорных векторов для наиболее оптимального разделения параметров на классы используюя несколько реализаций: линейную (LinearSVC) и через градиентный бустинг (SGDClassifier).
2. Сделать предсказание и проверить качество через каппа-метрику.

## Подключение библиотек

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score, confusion_matrix
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn import preprocessing

from core.reduce_mem_usage import reduce_mem_usage

## Загрузка данных

In [2]:
data = pd.read_csv("../data/prudential/train.csv.gz")
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59381 entries, 0 to 59380
Columns: 128 entries, Id to Response
dtypes: float64(18), int64(109), object(1)
memory usage: 58.0+ MB
None


## Предобработка данных

In [3]:
data["Product_Info_2_1"] = data["Product_Info_2"].str.slice(0, 1)
data["Product_Info_2_2"] = pd.to_numeric(data["Product_Info_2"].str.slice(1, 2))
data.drop(labels=["Product_Info_2"], axis=1, inplace=True)

In [4]:
list_df = [data]
for un in data["Product_Info_2_1"].unique():
    s = pd.DataFrame()
    s["Product_Info_2_1_" + un] = data["Product_Info_2_1"].isin([un]).astype("int8")
    list_df.append(s)
data = pd.concat(list_df, axis=1)
data.drop(labels=["Product_Info_2_1"], axis=1, inplace=True)

In [5]:
data.fillna(value=-1, inplace=True)
print(data.head())

   Id  Product_Info_1  Product_Info_3  Product_Info_4  Product_Info_5  \
0   2               1              10        0.076923               2   
1   5               1              26        0.076923               2   
2   6               1              26        0.076923               2   
3   7               1              10        0.487179               2   
4   8               1              26        0.230769               2   

   Product_Info_6  Product_Info_7   Ins_Age        Ht        Wt  ...  \
0               1               1  0.641791  0.581818  0.148536  ...   
1               3               1  0.059701  0.600000  0.131799  ...   
2               3               1  0.029851  0.745455  0.288703  ...   
3               3               1  0.164179  0.672727  0.205021  ...   
4               3               1  0.417910  0.654545  0.234310  ...   

   Medical_Keyword_46  Medical_Keyword_47  Medical_Keyword_48  Response  \
0                   0                   0            

## Набор столбцов для расчета

In [6]:
columns_groups = [
    "Insurance_History",
    "InsuredInfo",
    "Medical_Keyword",
    "Family_Hist",
    "Medical_History",
    "Product_Info",
]
columns = [
    "Wt",
    "Ht",
    "Ins_Age",
    "BMI",
]

for cg in columns_groups:
    columns.extend(data.columns[data.columns.str.startswith(cg)])
print(columns)

['Wt', 'Ht', 'Ins_Age', 'BMI', 'Insurance_History_1', 'Insurance_History_2', 'Insurance_History_3', 'Insurance_History_4', 'Insurance_History_5', 'Insurance_History_7', 'Insurance_History_8', 'Insurance_History_9', 'InsuredInfo_1', 'InsuredInfo_2', 'InsuredInfo_3', 'InsuredInfo_4', 'InsuredInfo_5', 'InsuredInfo_6', 'InsuredInfo_7', 'Medical_Keyword_1', 'Medical_Keyword_2', 'Medical_Keyword_3', 'Medical_Keyword_4', 'Medical_Keyword_5', 'Medical_Keyword_6', 'Medical_Keyword_7', 'Medical_Keyword_8', 'Medical_Keyword_9', 'Medical_Keyword_10', 'Medical_Keyword_11', 'Medical_Keyword_12', 'Medical_Keyword_13', 'Medical_Keyword_14', 'Medical_Keyword_15', 'Medical_Keyword_16', 'Medical_Keyword_17', 'Medical_Keyword_18', 'Medical_Keyword_19', 'Medical_Keyword_20', 'Medical_Keyword_21', 'Medical_Keyword_22', 'Medical_Keyword_23', 'Medical_Keyword_24', 'Medical_Keyword_25', 'Medical_Keyword_26', 'Medical_Keyword_27', 'Medical_Keyword_28', 'Medical_Keyword_29', 'Medical_Keyword_30', 'Medical_Keywor

## Нормализация данных

In [7]:
scaler = preprocessing.StandardScaler()
data_transformed = pd.DataFrame(scaler.fit_transform(pd.DataFrame(data, columns=columns)))
columns_transformed = data_transformed.columns
data_transformed["Response"] = data["Response"]

## Оптимизация памяти

In [8]:
data_transformed = reduce_mem_usage(data_transformed)
print(data_transformed.info())

Потребление памяти меньше на - 42.87 Мб (минус 75.1%)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59381 entries, 0 to 59380
Columns: 126 entries, 0 to Response
dtypes: float16(125), int8(1)
memory usage: 14.2 MB
None


## Разделение данных

In [9]:
data_train, data_test = train_test_split(data_transformed, test_size=0.2)
print(data_train.head())

              0         1         2         3         4         5         6  \
45829  0.261719  0.024353  0.289551  0.347412 -1.634766 -0.169434  0.862305   
43227  0.003372 -0.465576  0.970703  0.353271  0.611816 -0.169434 -1.159180   
40043  1.130859  0.269287  0.138306  1.240234  0.611816 -0.169434  0.862305   
25927 -0.818848 -0.955078  1.727539 -0.447998 -1.634766 -0.169434  0.862305   
36567 -0.161133  0.759277 -1.375977 -0.607422  0.611816 -0.169434 -1.159180   

              7         8         9  ...       116      117       118  \
45829 -1.013672  0.864258 -0.928711  ... -0.083679  0.44165 -0.149292   
43227  1.100586 -1.156250  1.130859  ... -0.083679  0.44165 -0.149292   
40043 -1.013672  0.864258 -0.928711  ... -0.083679  0.44165 -0.149292   
25927 -1.013672  0.862793 -0.928711  ... -0.083679  0.44165 -0.149292   
36567  1.100586 -1.156250  1.130859  ... -0.083679  0.44165 -0.149292   

            119       120       121       122       123      124  Response  
45829 -0.

## SVM

In [11]:
x = pd.DataFrame(data_train, columns=columns_transformed)
model_lin = LinearSVC(max_iter=10000)
model_lin.fit(x, data_train["Response"])

LinearSVC(max_iter=10000)

In [11]:
model_sgd = SGDClassifier()
model_sgd.fit(x, data_train["Response"])

SGDClassifier()

## Предсказание данных и оценка модели

In [12]:
x_test = pd.DataFrame(data_test, columns=columns_transformed)
data_test["target_lin"] = model_lin.predict(x_test)
data_test["target_sgd"] = model_sgd.predict(x_test)

In [13]:
print(f'SVM (линейный): {round(cohen_kappa_score(data_test["target_lin"], data_test["Response"], weights="quadratic"), 3)}')
print(f'SVM (градиент): {round(cohen_kappa_score(data_test["target_sgd"], data_test["Response"], weights="quadratic"), 3)}')

SVM (линейный): 0.471
SVM (градиент): 0.442


## Матрица неточностей

In [15]:
print(f'SVM (линейный):\n\n {confusion_matrix(data_test["target_lin"], data_test["Response"])}', end="\n\n")
print(f'SVM (градиент):\n\n {confusion_matrix(data_test["target_sgd"], data_test["Response"])}')

SVM (линейный):

 [[ 304  163   13    6   64  132   50   31]
 [ 174  288   12    3  107  126   17    8]
 [   0    0    0    1    0    0    0    0]
 [   7    7   10   37    0   13    1    1]
 [  78  150   13    0  270   97   33    9]
 [ 255  245  140  171  240  919  192  144]
 [ 129  109    3    4  124  253  425   53]
 [ 338  316   20   59  279  716  841 3677]]

SVM (градиент):

 [[ 295  262   22   19  121  205   96   57]
 [ 154  175   19   16  103  185   85   40]
 [  12    6   24   21    9   30    5    5]
 [  26   24   19   71   11  113   18   24]
 [ 133  183   17    7  279  195   98   73]
 [ 303  278   88   91  262  818  354  167]
 [  72   76    7   12   73  161  201   87]
 [ 290  274   15   44  226  549  702 3470]]
